In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from model import *

unlabel_prob = 0.9 # probability of labels to be unlabeled

sigma = 3          # sigma hyperparameter for RBF kernel

%matplotlib inline

### Cora Data

In [2]:
true_labels, features, graph = Data.Data.load_cora(multiclass=True)
_, is_labeled, labeled_indices, unlabeled_indices = utils.random_unlabel(true_labels,unlabel_prob=unlabel_prob)
solution  = true_labels[unlabeled_indices]
weights   = utils.rbf_kernel(features,s=sigma,G=graph)

### Data Preparation

In [3]:
lp = LP.LP()
lp_closed_prediction = np.rint(lp.iter_multiclass(labels,
                                  weights,
                                  labeled_indices,
                                  unlabeled_indices))

NameError: name 'labels' is not defined

In [8]:

def _tnorm(weights):
    '''
    Column normalize -> row normalize weights.
    '''
    # column normalize weights
    T = weights / np.sum(weights, axis=0, keepdims=True)
    # row normalize T
    Tnorm = T / np.sum(T, axis=1, keepdims=True)
    return Tnorm

def iter(X, # input labels
               weights,
               labeled_indices,
               unlabeled_indices,
               num_iter):
    '''
    Iterated solution of label propagation.
    '''
    # normalize T
    Tnorm = _tnorm(weights)
    h = X.copy()

    for i in range(num_iter):
        # propagate labels
        h = np.dot(Tnorm,h)
        # don't update labeled nodes
        h[labeled_indices] = X[labeled_indices]

    # only return label predictions
    return(h[unlabeled_indices])

def closed(labels,
                 weights,
                 labeled_indices,
                 unlabeled_indices):
    '''
    Closed solution of label propagation.
    '''
    # normalize T
    Tnorm = _tnorm(weights)
    # sort Tnorm by unlabeled/labeld
    Tuu_norm = Tnorm[np.ix_(unlabeled_indices,unlabeled_indices)]
    Tul_norm = Tnorm[np.ix_(unlabeled_indices,labeled_indices)]
    # closed form prediction for unlabeled nodes
    lapliacian = (np.identity(len(Tuu_norm))-Tuu_norm)
    propagated = Tul_norm @ labels[labeled_indices]
    label_predictions = np.linalg.solve(lapliacian, propagated)
    return label_predictions

def iter_multiclass(X, # input labels
                          weights,
                          labeled_indices,
                          unlabeled_indices,
                          num_iter=-1):
    preds = []
    num_classes = len(set(X))
    for class_ in range(num_classes):
        X_class = X.copy()
        X_class[labeled_indices] = X_class[labeled_indices] == class_
        X_class[unlabeled_indices] = np.array([1/num_classes] * len(unlabeled_indices))
        if num_iter == -1:
            pred = closed(X_class,weights,labeled_indices,unlabeled_indices)
        else:
            pred = iter(X_class,weights,labeled_indices,unlabeled_indices,num_iter)
        preds.append(pred)
    res = np.vstack(preds).T
    return res

In [18]:
for i in [1,2,3,4,5,10,100,-1]:
    pred = iter_multiclass(true_labels, # input labels
                              weights,
                              labeled_indices,
                              unlabeled_indices,
                              num_iter=i)
    print(np.sum(pred,axis=))
    Uy_pred_onehot = utils.prob_to_one_hot(pred)
    Uy_sol_onehot = utils.array_to_one_hot(solution)
    print(i)
    print(utils.accuracy_mult(Uy_sol_onehot,Uy_pred_onehot))

[ 0.          0.          0.06308234 ...,  0.34802518  0.16657572
  0.1324855 ]
1
0.262075134168
[ 0.09175801  0.03425075  0.12622617 ...,  0.34802518  0.286591    0.1412934 ]
2
0.620751341682
[ 0.1039972   0.06953249  0.18496836 ...,  0.59415316  0.35431877
  0.24274475]
3
0.722719141324
[ 0.17229029  0.10795475  0.24032337 ...,  0.63874581  0.42923369
  0.25592798]
4
0.748211091234
[ 0.19293468  0.1456399   0.28739013 ...,  0.75020238  0.47101466
  0.33431542]
5
0.75313059034
[ 0.36900834  0.32434067  0.45839839 ...,  0.89355704  0.66894915
  0.49201102]
10
0.764758497317
[ 0.91701738  0.96112796  0.97858434 ...,  0.99838752  0.99593156
  0.97126726]
100
0.780858676208
[ 1.  1.  1. ...,  1.  1.  1.]
-1
0.780858676208


(2236,)

In [10]:
solution.shape

(2236,)

In [38]:
Uy_pred_prob = lp.iter_multiclass(weights,Ly,num_classes,num_unlabeled,iter_=1)
Uy_pred_onehot = prob_to_one_hot(Uy_pred_prob)
Uy_sol_onehot = array_to_one_hot(Uy_sol,num_unlabeled,num_classes)
accuracy_mult(Uy_sol_onehot,Uy_pred_onehot)

0.004469727752945957

In [39]:
np.sum(Uy_pred_prob,axis=1)

array([ 1.66300126,  1.59990757,  0.24730996, ...,  1.37724886,
        1.87021331,  1.77064891])

In [5]:
X = []
Y = []
test_X = []
test_Y = []
for class_ in range(num_classes):
    Ly_class = Ly == class_
    Uy_class = np.array([1/num_classes] * num_unlabeled)
    Uy_sol_class = Uy_sol == class_
    
    Xc = np.hstack((Ly_class.astype(float),Uy_class)).reshape(1,num_nodes)
    XC = np.tile(Xc,(Ly_class.shape[0],1))
    np.fill_diagonal(XC, 1/num_classes)
    
    Yc = np.hstack((Ly_class.astype(float),Uy_sol_class)).reshape(1,num_nodes)
    YC = np.tile(Yc,(Ly_class.shape[0],1))
    
    test_X.append(Xc)
    test_Y.append(Yc)
    X.append(XC)
    Y.append(YC)
X = np.vstack(X)
Y = np.vstack(Y)
test_X = np.vstack(test_X)
test_Y = np.vstack(test_Y)

In [6]:
unlabeled_ = np.tile(np.hstack((np.identity(num_labeled),np.ones((num_labeled,num_unlabeled)))),(num_classes,1))
unlabeled_test = np.hstack((np.zeros(num_labeled),np.ones(num_unlabeled))).reshape(1,num_nodes)
labeled_ = 1-unlabeled_
true_labeled = np.array([1] * num_labeled + [0] * num_unlabeled).reshape(1,num_nodes)
masked_ = np.tile(np.hstack((np.identity(num_labeled),np.zeros((num_labeled,num_unlabeled)))),(num_classes,1))

In [7]:
test_data = {
    'X': test_X,
    'y': test_Y,
    'unlabeled': unlabeled_test,
    'labeled': (1 - unlabeled_test).reshape(1,num_nodes),
    'true_labeled': true_labeled,
    'masked':masked_[0:1]
}

In [8]:
data = {
    'X':X,
    'y':Y,
    'unlabeled':unlabeled_,
    'labeled':labeled_,
    'true_labeled': true_labeled,
    'masked':masked_
}

In [18]:


def dense_to_sparse(a_t):
    idx = tf.where(tf.not_equal(a_t, 0))
    # Use tf.shape(a_t, out_type=tf.int64) instead of a_t.get_shape() if tensor shape is dynamic
    sparse = tf.SparseTensor(idx, tf.gather_nd(a_t, idx), a_t.get_shape())
    return sparse

def init_weights(phi, G, sigma):
    r = tf.reduce_sum(phi*phi, 1)
    r = tf.reshape(r, [-1, 1])
    D = tf.cast(r - 2*tf.matmul(phi, tf.transpose(phi)) + tf.transpose(r),tf.float32)
    W = tf.SparseTensor.__mul__(G,tf.exp(-tf.divide(D, sigma ** 2)))
    return W


def calc_loss(mask,y,yhat):
    loss_mat = tf.multiply(mask, (y-yhat) ** 2 )
    return tf.reduce_sum(loss_mat) / tf.count_nonzero(loss_mat,dtype=tf.float32)

def calc_accuracy(y,prob,full=False):


    print(y,prob)
    yhat = tf.to_float(tf.equal(prob,tf.reduce_max(prob,axis=1)))
    print(yhat)
    return tf.reduce_all(tf.equal(yhat,y),axis=1)
     




In [19]:
phi         = tf.constant(features, dtype=tf.float32)
G           = dense_to_sparse(tf.constant(graph, dtype=tf.float32))
sigma  = tf.Variable(sigma, dtype=tf.float32)
W           = init_weights(phi, G, sigma)


lr     = 0.1
iter_  = 1 # Layer size

shape             = [None, num_nodes]
X            = tf.placeholder("float", shape=shape)
y            = tf.placeholder("float", shape=shape)
unlabeled    = tf.placeholder("float", shape=shape)
labeled      = tf.placeholder("float", shape=shape)
masked       = tf.placeholder("float", shape=shape)
true_labeled = tf.placeholder("float", shape=shape)

T = W / tf.sparse_reduce_sum(W, axis = 0, keep_dims=True)
Tnorm = T / tf.sparse_reduce_sum(T, axis = 1, keep_dims=True)

trueX = X
X = X

for i in range(iter_):
    h = tf.transpose(tf.sparse_tensor_dense_matmul(
                    Tnorm,
                    X,
                    adjoint_a=True,
                    adjoint_b=True,
    ))
    h = tf.multiply(h, unlabeled) + tf.multiply(trueX, labeled)
    X = h
yhat = X



# Backward propagation

loss = calc_loss(masked,y,yhat)
updates = tf.train.AdamOptimizer(lr).minimize(loss)

unlabeled_loss     = calc_loss((1-true_labeled),y,yhat)
accuracy           = calc_accuracy(y,yhat)
sol_unlabeled_loss = calc_loss((1-true_labeled),y,yhat)
sol_accuracy       = calc_accuracy(y,yhat,True)


Tensor("Placeholder_7:0", shape=(?, 2485), dtype=float32) Tensor("add_102:0", shape=(?, 2485), dtype=float32)
Tensor("ToFloat_2:0", shape=(?, 2485), dtype=float32)
Tensor("Placeholder_7:0", shape=(?, 2485), dtype=float32) Tensor("add_102:0", shape=(?, 2485), dtype=float32)
Tensor("ToFloat_3:0", shape=(?, 2485), dtype=float32)


In [24]:

sess   = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

def close_sess():
    sess.close()

def labelprop(data):
    open_sess()
    pred = eval(yhat,data)
    # self.close_sess()

    return pred

def eval(vals,data):
    return sess.run(vals, feed_dict={X:data['X'],
                                y:data['y'],
                                unlabeled:data['unlabeled'],
                                labeled:data['labeled'],
                                masked:data['masked'],
                                true_labeled:data['true_labeled']})

def save_params(epoch,data,n):
    pass

def save(epoch,data,full_data,n):

    print(eval([loss,unlabeled_loss,accuracy],data))
    print(eval([sol_accuracy,sol_unlabeled_loss],full_data))
    save_params(epoch,data,n)

def train(data,full_data,epochs):
    open_sess()

    n = len(data['X'])
    labeled_losses = []
    unlabeled_losses = []
    accuracies = []
    sol_accuracies = []
    sol_unlabeled_losses = []
    save(-1,data,full_data,n)
    for epoch in range(epochs):
        # Train with each example
        eval(updates,data)
        # print("--- %s seconds ---" % (time.time() - self.start_time))
        # self.start_time = time.time()
        save(epoch,data,full_data,n)
    # self.close_sess()


In [25]:
train(data,test_data,1)

InvalidArgumentError: Incompatible shapes: [168,2485] vs. [168]
	 [[Node: Equal_4 = Equal[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_add_102_0, Max_2)]]

Caused by op 'Equal_4', defined at:
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    sup

## Sparse

In [9]:
dlp = DeepLP_RBF_Sparse(100, num_nodes, features, graph, sigma, lr=0.01,print_freq=1,multi_class=True)

Tensor("Placeholder_1:0", shape=(?, 2485), dtype=float32) Tensor("add_100:0", shape=(?, 2485), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 2485), dtype=float32) Tensor("add_100:0", shape=(?, 2485), dtype=float32)


In [10]:
pred_dlp = dlp.labelprop(test_data)
Uy_pred_onehot = prob_to_one_hot(pred_dlp[:,num_labeled:].T)
Uy_sol_onehot = array_to_one_hot(Uy_sol,num_unlabeled,num_classes)
accuracy_mult(Uy_sol_onehot,Uy_pred_onehot)

0.16131653799268589

In [11]:
dlp.train(data,test_data,100)

InvalidArgumentError: Incompatible shapes: [168,2485] vs. [168]
	 [[Node: Equal = Equal[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](add_100, Max)]]

Caused by op 'Equal', defined at:
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, 

In [12]:
data['true_labeled'].shape

(1, 2485)